In [3]:
import pandas as pd
import numpy as np
from collections import Counter
import seaborn as sns 
import matplotlib.pyplot as plt

In [5]:
leiden=pd.read_csv('/Users/Evelyn/Documents/UKB/materials/leidenA.csv')
k3=pd.read_csv('/Users/Evelyn/Documents/UKB/materials/clust_3kA.csv')
comp=pd.read_csv('/Users/Evelyn/Documents/UKB/materials/clusterv_compA.csv')
clust2=pd.read_csv('/Users/Evelyn/Documents/UKB/materials/clusterv2A.csv').merge(leiden[['eid']])
dftot = pd.read_csv('/Users/Evelyn/Documents/UKB/materials/3k_prot_v1.csv').iloc[:,:-2].merge(leiden[['eid']])
df = pd.read_csv('/Users/Evelyn/Documents/UKB/materials/3kprot_all_expression_new.csv')

In [6]:
# Define loose proteins 
def prot_thresh(n, cluster_list):
    prot_lists = []
    for cl in cluster_list:
        prot_lists.extend(cl['protein'].tolist())
    prot_count = Counter(prot_lists)
    loose_prot = [pro for pro, count in prot_count.items() if count >= n] 
    return loose_prot

# Remove the proteins with low correlation
def remove_corr(dataframe, badflag, threshold, tokeep=None):
    def update_lost(lost, l, w):
        if w in lost.keys():
            if l in lost.keys():
                l_var=lost.pop(l)
                lost[w]=lost[w]+ l_var+[l]
            else:
                lost[w]=lost[w]+ [l]
        else:
            if l in lost.keys():
                l_var=lost.pop(l)
                lost[w]= l_var+[l]
            else:
                lost[w]= [l]
        return lost

    listvar=dataframe.columns.tolist()
    listvar.remove(badflag)
    listvar+=[badflag]
    dataframe=dataframe[listvar]
    corr_M=np.asmatrix(dataframe.corr())
    corr_M=abs(corr_M)
    bad=corr_M[:,-1].tolist()
    M_size=corr_M.shape
    bad=bad[:M_size[0]]+bad[M_size[0]:]
    corr_b=np.delete(corr_M, M_size[0]-1, axis=0)
    corr_M=np.delete(corr_b,M_size[1]-1, axis=1)
    for index in range(corr_M.shape[0]):
        corr_M[index,index]=0
    stopcondition=(corr_M>threshold).sum()
    lost={}
    while stopcondition>0:
        flat_index=corr_M.argmax()
        i=flat_index % corr_M.shape[0]
        j=int((flat_index - i )/corr_M.shape[0])
        if bad[i]>bad[j]:
            w,l=listvar[i],listvar[j]
            todelete=j
            altdelete=i
        else:
            l,w=listvar[i],listvar[j]
            todelete=i
            altdelete=j
    
        if tokeep:
            if l in tokeep:
                if w in tokeep:
                    corr_M[i,j]=0
                    corr_M[j,i]=0
                else:
                    l,w=w,l
                    
                    lost=update_lost(lost, l, w)
                    corr_M=np.delete(corr_M, altdelete, axis=0)
                    corr_M=np.delete(corr_M, altdelete, axis=1)
                    bad.pop(altdelete)
                    listvar.pop(altdelete)
            else:
                lost=update_lost(lost, l, w)
                corr_M=np.delete(corr_M, todelete, axis=0)
                corr_M=np.delete(corr_M, todelete, axis=1)
                bad.pop(todelete)
                listvar.pop(todelete)
        else:
            lost=update_lost(lost, l, w)
            corr_M=np.delete(corr_M, todelete, axis=0)
            corr_M=np.delete(corr_M, todelete, axis=1)
            bad.pop(todelete)
            listvar.pop(todelete)
        stopcondition=(corr_M>threshold).sum()
    return dataframe[listvar],lost

# Create heatmap by using correlation differences between whole population and within clusters
def heatmap(protein_list, incluster_people, dataset, new_list):
    inclust_prot = dataset[['eid'] + protein_list].merge(incluster_people, on='eid')
    inclust_prot['bad'] = 0 
    a,b=remove_corr(inclust_prot,'bad', 0.8)  
    corr={}
    for k in b.keys():
        if len([k]+b[k])>4: 
            corr[k]=[k]+b[k]
    for key, pro in corr.items():
        dfs = inclust_prot[pro].copy()
        correlation = dfs.corr(method='pearson')
        mean = correlation.mean().mean()

    inclust_corr = inclust_prot.iloc[:,1:].corr(method='spearman')
    mylist = []
    for key, pro in corr.items():
        mylist += pro 
    reorder1 = inclust_corr.reindex(index=mylist, columns=mylist) 
    all_corr = dataset[['eid']+protein_list].iloc[:,1:].corr(method='spearman')
    reorder2 = all_corr.reindex(index=mylist, columns=mylist)
    
    diff = reorder1-reorder2
    new = diff.reindex(index=new_list, columns=new_list)
    mini=min(reorder2.min().min(),reorder1.min().min(),(diff).min().min())
    maxi=max(reorder2.max().max(),reorder1.max().max())

    fig, ax = plt.subplots(figsize=(100, 100))
    sns.heatmap(new, cmap='RdBu', vmin=mini, vmax=maxi, center=0,annot=False, annot_kws={"size": 13}, ax=ax,
                cbar_kws={'shrink': 1, 'location': 'right', 'pad': 0.05})
    ax.set_xticklabels([])
    ax.set_yticklabels([])  
    ax.tick_params(bottom=False, left=False)
    cbar = ax.collections[0].colorbar
    if cbar:
        cbar.ax.tick_params(labelsize=150)
    plt.show()

    print(f"Min:{diff.min().min():.4f}")
    print(f"Max:{diff.max().max():.4f}")   

    return fig

## Figure5 A

In [6]:
# Loose common proteins in leukemia clusters
l1 = df[(df['file']=='comp')&(df['cluster']=='cl5')]
l2 = df[(df['file']=='comp')&(df['cluster']=='cl7')]
l3 = df[(df['file']=='cluster2')&(df['cluster']=='cl1')]
l4 = df[(df['file']=='cluster2')&(df['cluster']=='cl5')]
l5 = df[(df['file']=='3k')&(df['cluster']=='cl16')]
l6 = df[(df['file']=='leiden')&(df['cluster']=='cl10')]
loose_prot = prot_thresh(3, [l1, l2, l3, l4, l5, l6])

In [7]:
# Participants in leukemia clusters
inle1 = comp[comp['cl5']==1]
inle2 = comp[comp['cl7']==1]
inle3 = clust2[clust2['cl1']==1]
inle4 = clust2[clust2['cl5']==1]
inle5 = k3[k3['cl16']==1]
inle6 = leiden[leiden['cl10']==1] 
incl_leuk = inle1[['eid']].merge(inle2[['eid']], how='outer').merge(inle3[['eid']], how='outer').merge(inle4[['eid']], how='outer').merge(inle5[['eid']], how='outer').merge(inle6[['eid']], how='outer')

In [8]:
new_list = ['mnat1', 'snu13','arid4b', 'gpkow', 'tdp1', 'rfc4', 'morc3','raly', 'tpr', 'cwc15', 'smndc1', 'npm1', 'nbn', 'eloa', 'zhx2', 'snrpb2',
            'bap18','zbtb17', 'setmar', 'c7orf50','sart1', 'pqbp1', 'lyar', 'tcof1', 'eri1', 'nfya', 'dgcr6', 'glyr1', 'myo9b','foxj3', 
             'ing1','polr2f', 'hdgfl2','kctd5', 'rnaseh2a','psip1', 'septin9', 'fgr', 'wdr46','lrch4', 'hnrnpul1', 'azi2', 'lrrfip1', 'sdccag8', 'pibf1', 'ptpn1', 'xiap', 'vamp8', 'calcoco2', 'tax1bp1', 'gigyf2', 'prkar2a', 'eef1d',
            'stx8', 'sugp1', 'dnmbp', 'coro1a', 'mzt1', 'cep170', 'ahcy', 'msra', 'gpi', 'plin3', 'hspa1a', 'vti1a', 'spart', 
            'samd9l', 'inpp1', 'taldo1', 'pgd', 'casp8', 'washc3', 'fgd3', 'eif2ak2', 'dnajb14', 'bag4', 'anxa4', 'commd1', 
            'nagk', 'dpy30', 'cep20', 'becn1', 'atp6v1g1', 'paxx', 'nub1', 'mphosph8', 'top2b', 'dgka', 'cetn2','wwp2', 'tnip1', 'rnf41', 'pfkfb2','tbc1d5', 
            'lbr', 'stx4', 'serpinb1', 'tnfaip2', 'irag2', 'hmox2', 'sh3bp1', 'ddx58', 'pag1', 'cracr2a', 'sptlc1', 
             'dapk2', 'nampt',  'dynlt1',  'aif1', 'egln1', 'casp10', 'ceacam8','rassf2',  'zbp1','tigar', 'apbb1ip',  'pxn','nedd9','azu1', 'rnase3',
            'mpo', 'serpinb8', 'nadk', 'prtn3', 'defa1_defa1b', 'anxa1', 'rab44','casp1', 's100a11', 'rbp7', 
            'bcl2l15', 's100p', 'capg',  'lpcat2', 'padi4', 'nmnat1', 'mnda', 'cda','il16', 'fmnl1','fen1', 'cep152', 'trim26','mapk13' ]
fig = heatmap(loose_prot,incl_leuk, dftot, new_list)
fig.savefig('Fig5_A.png', dpi=100, bbox_inches='tight')

Min:-0.2992
Max:0.7292


## Figure5 B

In [10]:
# Loose common proteins in hypeertension clusters
hy1 = df[(df['file']=='leiden')&(df['cluster']=='cl1')] 
hy2 = df[(df['file']=='leiden')&(df['cluster']=='cl2')] 
hy3 = df[(df['file']=='leiden')&(df['cluster']=='cl5')] 
hy4 = df[(df['file']=='leiden')&(df['cluster']=='cl6')]
loose_prot = prot_thresh(3, [hy1, hy2, hy3, hy4])

In [11]:
# Participants in hypertension clusters
inhy1 = leiden[leiden['cl1']==1] 
inhy2 = leiden[leiden['cl2']==1] 
inhy3 = leiden[leiden['cl5']==1] 
inhy4 = leiden[leiden['cl6']==1]   
incl_hyp = inhy1[['eid']].merge(inhy2[['eid']], how='outer').merge(inhy3[['eid']], how='outer').merge(inhy4[['eid']], how='outer')

In [12]:
new_list = ['acadsb', 'sytl4', 'timm10', 'rab10', 'lrrc59', 'jam3', 'mvk', 'hmgcl', 'echs1', 'cox5b', 'clpp', 'chm', 'casc3', 'ptges2', 
            'grsf1', 'arsb', 'siae', 'selp', 'scpep1', 'sema4d', 'scarf1', 'tmem106a', 'cd63', 'erp44', 'anxa11','hspb1', 'serpinb9', 
            'susd1', 'arhgap1', 'aamdc', 'dnajc9', 'urod', 'atxn3', 'rilp', 'impact', 'ppcdc', 'pdcd5', 'snca', 'tp53i3', 'bola2_bola2b', 
            'psme1', 'nit1', 'sod1', 'eif4ebp1', 'ppme1', 'atg4a', 'add1', 'taldo1', 'qdpr', 'kyat1', 'lgals8', 'syap1', 'plscr3', 'ipcef1', 'trim5', 'foxo1', 
            'casp2', 'cep43', 'ankmy2', 'nexn', 'zbtb16', 'birc2', 'pacs2', 'tax1bp1', 'tbc1d5', 'cracr2a', 'cep170', 'map3k5', 'camsap1', 
            'azi2', 'rab33a', 'tnfaip2', 'stk24', 'metap2', 'pebp1', 'stip1', 'park7', 'nsfl1c', 'glod4', 'gmpr2', 'igbp1', 'mdh1', 'dnajb2',
            'st13', 'dtymk', 'dnajb1', 'dnajc6', 'fhit', 'rabgap1l', 'fkbp4', 'hgs', 'comt', 's100a4', 'nit2', 'grhpr', 'ggct', 'fkbpl', 
            'vta1', 'akt1s1', 'coro1a', 'akt3', 'cetn2', 'stx4', 'tymp', 'plin3', 'nfkb1', 'lrrfip1', 'bach1', 'fam13a', 'cep85', 'fabp5',
            'inpp1', 'bag4', 'mocs2', 'p4hb', 'nfu1', 'mtif3', 'ndufb7', 'erp29', 'dctd', 'mars1', 'dbnl', 'crkl', 'nck2', 'ppp1r2', 'grap2',
            'casp3', 'rilpl2', 'dnajb6', 'gcc1', 'mesd', 'msra', 'hars1', 'tpd52l2', 'mitd1', 'cacybp', 'tmed8', 'pde5a', 'pdlim5', 'dok1', 
            'twf2', 'gmfg', 'ist1', 'bcl2l1', 'tbcb', 'stat5b', 'tbcc', 'map4k5', 'nrgn', 'jpt2', 'irak4', 'ufd1', 'ywhaq', 'slc9a3r1', 
            'mydgf', 'yars1', 'cirbp', 'med18', 'plcb2', 'dtd1', 'naa80', 'hpcal1', 'gipc3', 'phactr2', 'pstpip2', 'vasp', 'erbin', 'eif4g1',
            'dnaja2', 'ebag9', 'fkbp5', 'lyn', 'yes1', 'lat2', 'src', 'pla2g4a', 'snap29', 'pdlim7', 'dok2', 'tia1', 'mink1', 'dab2', 
            'prtfdc1', 'vav3', 'map2k6', 'elac1', 'itgb1bp2', 'nap1l4', 'acyp1', 'vps53', 'srpk2', 
            'wwp2', 'ubxn1', 'eif4b', 'snx9', 'cnp', 'cc2d1a', 'samd9l', 'fam172a', 'commd1', 'calcoco2', 'wasf1', 'ptpn1', 'dnmbp', 'smad1',
            'vash1', 'fli1', 'hhex', 'cdkn1a', 'tarbp2', 'irak1', 'ica1', 'ppp1r9b', 'map2k1', 'cdc26', 'ap1g2', 'rnf41', 'gfer', 'pibf1', 
            'lats1', 'vps37a', 'dctn1', 'tdrkh', 'atp6v1g1', 'becn1', 'txndc5', 'vps4b', 'ppm1f', 'cmc1', 'casp7', 'gopc', 'ldlrap1', 'pcbp2', 'fgd3', 'mgll', 
            'ptpn6', 'rhoc', 'manf', 'ca13', 'sec31a', 'sarg', 'numb', 'tacc3', 'sla2', 'gtpbp2', 'hs1bp3', 'akt2', 'dut', 'triap1', 'fxn', 
            'rtn4ip1', 'mecr', 'lonp1', 'glrx5', 'metap1d', 'decr1', 'fkbp14', 'prkar2a', 'eif2ak2', 'cmip', 'oga', 'ddhd2', 'trim25',
            'atg16l1', 'stat2', 'vamp8', 'zfyve19', 'otud6b', 'arhgef1', 'evi5', 'dlg4', 'plekho1', 'xiap', 'bloc1s3', 'ap3b1', 'ak2',
            'snap23', 'diablo', 'ppib', 'gp6', 'clec1b', 'plxna4', 'mpig6b', 'cnst', 'cd69', 'atp5if1', 'rab11fip3', 'prkg1', 'mindy1',
            'spart', 'usp25', 'rabep1', 'arhgap45', 'tbc1d23', 'brap', 'rbpms2', 'rab27b', 'tjap1', 'ikbkg', 'inppl1', 'txlna', 'nfatc1',
            'bank1', 'axin1', 'bcr', 'kazn', 'ophn1', 'git1', 'scamp3', 'hexim1', 'mavs', 'tnip1', 'sdccag8', 'tab2', 'cdc42bpb', 'calcoco1',
            'arhgef12', 'uso1', 'nudc', 'nfe2', 'otud7b', 'ehbp1', 'washc3', 'eef1d', 'daam1', 'sh2b3', 'clip2', 'rgcc', 'tbl1x', 'kifbp', 
            'dapp1', 'pmvk', 'prdx5', 'cdkn2d', 'bin2', 'nt5c3a', 'dars1', 'appl2', 'tomm20', 'l3hypdh', 'cetn3', 'pcyt2', 'dnm1', 'mtdh', 
            'ppp2r5a', 'gadd45gip1', 'slmap', 'vti1a', 'esyt2', 'sult1a1', 'rab2b', 'nudt16', 'pdia4', 'gpi', 'ube2l6', 'crybb1', 'smad2',
            'thtpa', 'f2r', 'htra2', 'stk11', 'hspa1a', 'fis1', 'dag1', 'hnrnpk', 'dgka', 'chmp6', 'stxbp1', 'cacnb3', 'aifm1', 'chmp1a', 
            'max', 'rgs10', 'impa1', 'timm8a', 'fkbp1b', 'spry2', 'gorasp2', 'ehd3', 'gatd3', 'plxnb3', 'mcfd2', 'nars1', 'ggact', 'lat', 
            'hpse', 'treml1', 'egf', 'mgmt', 'lactb2', 'fadd', 'rwdd1', 'ciapin1', 'sirt2', 'stambp', 'cryzl1', 'tbca', 'ptrhd1', 'dffa', 
            'eif4e', 'plpbp', 'mif', 'cradd', 'cdc37', 'usp8', 'nub1', 'serpinb6', 'serpinb1', 'tmsb10', 'txnrd1', 'nudt5', 'dbi',
            'tnfaip8l2', 'lztfl1', 'stk4', 'eno2', 'eno1', 'pdap1', 'mpi', 'rras', 'ppp1cc', 'cnpy4', 'sugt1', 'drg2', 'anxa4', 'hcls1', 
            'pgd', 'casp8', 'pfkfb2', 'anxa3', 'dnajb14', 'prkar1a', 'rbm17', 'prdx3', 'mphosph8', 'tst', 'nt5c', 'serpinh1', 'oxct1',
            'atox1', 'mthfsd', 'txndc9', 'mapkapk2', 'gys1', 'acot13', 'dpy30', 'nfkbie', 'ap3s2', 'prkra', 'nagk', 'oplah', 'bag6', 
            'gigyf2', 'sugp1', 'top2b', 'paxx', 'hnrnpul1', 'hdgfl2', 'lbr', 'ahcy', 'trim58', 'aspscr1', 'ythdf3', 'rapgef2', 'arhgef10',
            'gne', 'mtss2', 'rnf5', 'asrgl1', 'reep4', 'lpp', 'cep20', 'dxo', 'arfip1', 'rrm2b', 'vps28', 'lrch4', 'nucb2', 'cert', 'skap2',
            'ppp1r12a', 'fyb1', 'f11r', 'vsir', 'cd40', 'abraxas2', 'stx8', 'bax', 'dbn1', 'cct5', 'fxyd5', 'tsc22d1', 'mtss1', 'frmd4b', 
            'rad23b', 'nfx1', 'inpp5d', 'cd2ap', 'myh9', 'arf6', 'prkab1', 'cdc27', 'ssna1', 'trim21', 'tank', 'rp2', 'abl1', 'aprt',
            'scrn1', 'abhd14b', 'pmm2', 'pnma1', 'smad3', 'gp1bb', 'arl2bp']
fig = heatmap(loose_prot, incl_hyp, dftot, new_list)
fig.savefig('Fig5_B.png', dpi=100, bbox_inches='tight')

Min:-0.0769
Max:0.3260


## Figure5 C

In [14]:
# Loose common proteins in celiac clusters
c1 = df[(df['file']=='comp')&(df['cluster']=='cl2')]
c2 = df[(df['file']=='comp')&(df['cluster']=='cl3')]
c3 = df[(df['file']=='3k')&(df['cluster']=='cl6')]
c4= df[(df['file']=='leiden')&(df['cluster']=='cl7')]
loose_prot = prot_thresh(2, [c1, c2, c3, c4])

In [15]:
# Participants in celiac clusters
ince1 = comp[comp['cl2']==1]
ince2 = comp[comp['cl3']==1]
ince3 = k3[k3['cl6']==1]
ince4 = leiden[leiden['cl7']==1] 
incl_celi = ince1[['eid']].merge(ince2[['eid']], how='outer').merge(ince3[['eid']], how='outer').merge(ince4[['eid']], how='outer')

In [16]:
new_list = ['ifit1', 'aida', 'rbp1', 'crtap', 'clec2l', 'cntnap4', 'pafah1b3', 'nenf', 'tdp1', 'spring1', 'nubp1', 'bcl7b', 'ugdh', 
            'sprr1b','palm3', 'ogt', 'gabarapl1', 'nrxn3', 'ecscr', 'txnl1', 'vstm2b', 'prkag3', 'cacnb1', 'snu13', 'igf2bp3', 
            'fdx1', 'med21', 'sh3gl3','nop56', 'bloc1s2','ttn',  'paip2b','stoml2','htr1b', 'myl6b','morf4l2', 'dcun1d1', 'mapre3',  
            'arhgap30', 'wfdc1', 'rbpms', 'clasp1', 'brd2', 'cwc15', 'stau1', 'pdrg1','suox',  'snx2','eif1ax','cabp2','amotl2', 'ift20',
            'rnaseh2a','sh3bp1','rpe', 'grsf1', 'sms', 'csde1', 'ak2', 'farsa', 'gorasp2', 'oplah', 'dlg4','snx5','eif2s2','dnajc21' ]
fig = heatmap(loose_prot, incl_celi, dftot, new_list)
fig.savefig('Fig5_C.png', dpi=100, bbox_inches='tight')

Min:-0.0174
Max:0.7117
